In [11]:
!pip install pycaret
! pip install flask-ngrok

In [12]:
%mkdir templates -p

In [13]:
%%writefile templates/home.html
<!DOCTYPE html>
<html lang="en">
<head>
  <title> Прогнозирование сердечной недостаточности </title>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.1/css/bootstrap.min.css">
  <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
  <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.1/js/bootstrap.min.js"></script>
</head>
<body>

<div class="container">
  <h1> Прогнозирование сердечной недостаточности </h1>
  <form action="{{ url_for('predict')}}" method = "POST">
    <div class="form-group">
      <label for="Age"> Введите возраст пациента: </label>
      <input type="Age" class="form-control" id="Age" placeholder="Введите возраст" name="Age" oninput="this.value">
      <label for="Anaemia"> Страдает ли пациент от анемии? Введите 0, если нет или 1, если страдает. </label>
      <input type="Anaemia" class="form-control" id="Anaemia" placeholder="Есть ли анемия?" name="Anaemia" oninput="this.value">
      <label for="CPK"> Уровень фермента КФК в крови (мкг / л) </label>
      <input type="CPK" class="form-control" id="CPK" placeholder="Введите уровень КФК" name="CPK" oninput="this.value">
      <label for="Diabetes"> Болен ли пациент диабетом? Введите 0, если нет или 1, если болен. </label>
      <input type="Diabetes" class="form-control" id="Diabetes" placeholder="Есть ли дабет?" name="Diabetes" oninput="this.value">
      <label for="Ejection_fraction"> Процент крови, покидающей сердце при каждом сокращении (в процентах) </label>
      <input type="Ejection_fraction" class="form-control" id="Ejection_fraction" placeholder="Введите процент крови, покидающий сердце при каждом сокращении." name="Ejection_fraction" oninput="this.value">
      <label for="Hypertension"> Есть ли у пациента гипертония? Введите 0, если нет или 1, если присутствует. </label>
      <input type="Hypertension" class="form-control" id="Hypertension" placeholder="Есть ли гипертония?" name="Hypertension" oninput="this.value">
      <label for="Platelets"> Тромбоциты в крови (килотромбоциты / мл) </label>
      <input type="Platelets" class="form-control" id="Platelets" placeholder="Введите количество тромбоцитов в крови. " name="Platelets" oninput="this.value">
      <label for="Serum_creatinine"> Уровень сыворотки креатинина в крови (мг / дл) </label>
      <input type="Serum_creatinine" class="form-control" id="Serum_creatinine" placeholder="Введите уровень креатинина." name="Serum_creatinine" oninput="this.value">
      <label for="Serum_sodium"> Уровень сывороточного натрия в крови (мг-экв / л) </label>
      <input type="Serum_sodium" class="form-control" id="Serum_sodium" placeholder="Введите уровень натрия." name="Serum_sodium" oninput="this.value">
      <label for="Sex"> Введите пол пациента.Введите 0, если женщина или 1, если мужчина. </label>
      <input type="Sex" class="form-control" id="Sex" placeholder="Пациент женщина или мужчина?" name="Sex" oninput="this.value">
      <label for="Smoking"> Курит ли пациент? Введите 0, если не курит или 1, если курит. </label>
      <input type="Smoking" class="form-control" id="Smoking" placeholder="Курит ли пациент?" name="Smoking" oninput="this.value">
      <label for="Times"> Срок наблюдения (дни)</label>
      <input type="Times" class="form-control" id="Times" placeholder="Сколько дней проводится наблюдение за больным?" name="Times" oninput="this.value">
    </div>
   
    <button type="submit" class="btn btn-success"> Умрёт ли пациент за время наблюдения? </button>
    <h2>{{ pred }}</h2>
  </form>
</div>

</body>
</html>

Overwriting templates/home.html


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, url_for, redirect, render_template, jsonify
from pycaret.regression import *
import pandas as pd
import pickle
import numpy as np

app = Flask(__name__)

run_with_ngrok(app)   

model = load_model('/content/drive/My Drive/Colab Notebooks/Heart_failure')
cols = ['age','anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time']

@app.route("/")
def home():
   return render_template("home.html")

@app.route("/predict", methods = ['POST'])
def predict():
  age = request.form['Age']
  anaemia = request.form['Anaemia']  
  cpk = request.form['CPK']
  diabetes = request.form['Diabetes']
  ejection_fraction = request.form['Ejection_fraction']
  hypertension = request.form['Hypertension']
  platelets = request.form['Platelets']
  serum_creatinine = request.form['Serum_creatinine']
  serum_sodium = request.form['Serum_sodium']
  sex = request.form['Sex']
  smoking = request.form['Smoking']
  times = request.form['Times']
  heart_failure_list = [age, anaemia, cpk, diabetes, ejection_fraction, hypertension, platelets, serum_creatinine, serum_sodium, sex, smoking, times]  
  
  sample_frame = pd.DataFrame([heart_failure_list] , columns = cols)
  prediction = predict_model(model, data = sample_frame, round = 0)
  prediction = prediction.Label[0]

  if prediction == 1:
    predict = "Всё плохо! Нужна срочная госпитализация!"
  else:
    predict = "Всё в порядке."

  return render_template('home.html', pred = predict)


if __name__ == '__main__':
    app.run()

Transformation Pipeline and Model Successfully Loaded
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://746976c878b0.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [27/Oct/2020 17:16:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2020 17:16:54] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Oct/2020 17:17:09] "POST /predict HTTP/1.1" 200 -
